In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from Bio import SeqIO
import glob
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [56]:
# If executing with snakemake, use the path below
sys.path.append(snakemake.config["WORKFLOW_PATH"]+'/snakemodules/notebooks/src/')
# If executing directly with jupyter notebook from the commandline, use the path below
# sys.path.append(os.getcwd()+'/src/')
from stat_func import calculator
from stat_func import visualization
%matplotlib inline

In [57]:
# # Get the current working directory
# print("Current Path:", current_path)
# # Navigate up two directories to reach '/home/usr/program'
# print("Program Path:", program_path)
# Join 'results' to get the desired path
# current_path = os.getcwd()
# program_path = os.path.dirname(os.path.dirname(current_path))
# results_path = os.path.join(program_path, 'results')

# If executing with snakemake, use the path below
results_path = os.path.join(os.getcwd(), 'results')
print("Results Path:", results_path)

In [63]:
# Use paths below for snakemake execution
supercontigs = snakemake.input.fasta
coverage_files = snakemake.input.cov

# Use paths below for jupyter notebook execution
# For jupyter notebook execution, supercontigs is a list after glob.glob
# supercontig_fa = glob.glob('/home/kedic/popinSnake/results/supercontigs.fa')
# supercontigs = supercontig_fa[0]
# coverage_files = glob.glob('/home/kedic/popinSnake/workdir/**/coverage.txt')

In [65]:
# For jupyter notebook execution, change supercontigs to supercontigs[0]
seq_objects=SeqIO.parse(supercontigs,'fasta')
sequences=[]
for seq in seq_objects:
    sequences.append(seq)

num_of_contigs = len(sequences)

In [16]:
contig_id = []
contig_length = []
for record in sequences:
    record_id = record.id
    sequence = record.seq
    length=len(sequence)
    contig_id.append(record_id)
    contig_length.append(length)

In [44]:
df_sort_contig = pd.DataFrame({
    'contig_id':contig_id,
    'contig_length':contig_length
})
df_sort_contig = df_sort_contig.sort_values(by=['contig_length'], ascending=True)
order_by_length = df_sort_contig['contig_id'].tolist()

In [90]:
# Initialize an empty DataFrame to hold all coverage data
coverage_df = pd.DataFrame()

for file in coverage_files:
    # Extract sample name from the filename
    folder_path = os.path.dirname(file)
    sample_name = os.path.basename(folder_path)
    # Read the coverage data, skipping comment lines starting with '#'
    df = pd.read_csv(file, sep='\t')
    # Select the contig name and mean depth columns
    df = df[['#rname', 'meandepth']]
    # Rename the mean depth column to the sample name
    df.rename(columns={'meandepth': sample_name}, inplace=True)
    if coverage_df.empty:
        # Initialize the main DataFrame with the first sample
        coverage_df = df
    else:
        # Merge with the main DataFrame on the contig name
        coverage_df = pd.merge(coverage_df, df, on='#rname', how='outer')


In [ ]:
coverage_df.rename(columns={'#rname': 'contigs'}, inplace=True)
coverage_df.set_index('contigs', inplace=True)

In [ ]:
coverage_df = coverage_df[coverage_df.index.str.startswith("contig_")]
coverage_df = coverage_df.reindex(order_by_length)

In [ ]:
coverage_df_cleaned = calculator.remove_outliers_replace_with_bounds(coverage_df)

In [85]:
# Plot the heatmap
sns.set(font_scale = 0.8)
sns.heatmap(coverage_df_cleaned, annot=False, cmap='YlOrBr')
plt.title('Coverage of Contigs')
plt.xlabel('Samples')
plt.ylabel('Contigs')

plt.savefig(snakemake.output[0], dpi=400, bbox_inches = 'tight')
plt.show()